In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification


In [ ]:
!pip install tensorflow

In [ ]:
!pip install transformers

In [ ]:
!pip install tf-keras

In [ ]:
!pip install scikit-learn

In [3]:
df = pd.read_csv('blooms_level Training.csv')

In [5]:
df['Questions'] = df['Questions'].str.replace(r'[^\w\s]+', '', regex=True)
df['Questions'] = df['Questions'].str.lower()



df['Label'] = pd.factorize(df.Label)[0]

categories = ['Remember', 'Understand', 'Apply', 'Analyse', 'Evaluate', 'Create']


df['Label'].value_counts()

print(df['Label'])

0      0
1      1
2      2
3      3
4      0
      ..
295    2
296    5
297    4
298    1
299    4
Name: Label, Length: 300, dtype: int64


In [7]:
train, val = train_test_split(df, test_size=0.2, random_state=0)

train['Label'].value_counts()

balanced__train = train

balanced__train['Label'].value_counts()

train_texts = list(balanced__train['Questions'])
train_labels = list(balanced__train['Label'])

val_texts = list(val['Questions'])
val_labels = list(val['Label'])

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [11]:
LEARNING_RATE = 0.000001
BATCH_SIZE = 24
EPOCHS = 50

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(train_dataset.batch(BATCH_SIZE), epochs=EPOCHS, batch_size=BATCH_SIZE,
          validation_data=val_dataset.batch(BATCH_SIZE), shuffle=True, callbacks=[callback])

category_dict = {0: 'remember', 1: 'understand', 2: 'apply', 3: 'analyse', 4: 'evaluate', 5: 'create'}

def predict_blooms(text):
    predict_input = tokenizer.encode(text,
                                     truncation=True,
                                     padding=True,
                                     return_tensors="tf")

    output = model(predict_input)[0]

    prediction_value = tf.argmax(output, axis=1).numpy()[0]

    return category_dict[prediction_value]



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/50


10/10 [==============================] - 23s 1s/step - loss: 1.7878 - accuracy: 0.1958 - val_loss: 1.7853 - val_accuracy: 0.1500
Epoch 2/50
10/10 [==============================] - 11s 1s/step - loss: 1.7839 - accuracy: 0.1875 - val_loss: 1.7797 - val_accuracy: 0.1500
Epoch 3/50
10/10 [==============================] - 11s 1s/step - loss: 1.7821 - accuracy: 0.1917 - val_loss: 1.7730 - val_accuracy: 0.1500
Epoch 4/50
10/10 [==============================] - 12s 1s/step - loss: 1.7708 - accuracy: 0.2583 - val_loss: 1.7656 - val_accuracy: 0.1500
Epoch 5/50
10/10 [==============================] - 11s 1s/step - loss: 1.7684 - accuracy: 0.2417 - val_loss: 1.7575 - val_accuracy: 0.2500
Epoch 6/50
10/10 [==============================] - 11s 1s/step - loss: 1.7594 - accuracy: 0.3000 - val_loss: 1.7479 - val_accuracy: 0.2833
Epoch 7/50
10/10 [==============================] - 11s 1s/step - loss: 1.7488 - accuracy: 0.3583 - val_loss: 1.7368 - val_accuracy: 0.4833
Epoch 8/50
10/10 [

In [13]:
test_text = "Design an algorithm to implement the A* search algorithm for pathfinding."

print(test_text)
print("Predicted Class:", predict_blooms(test_text))

Design an algorithm to implement the A* search algorithm for pathfinding.
Predicted Class: understand
